# Run a report from 3E
As we saw in the [Interacting with SQL](Interacting%20with%20SQL.ipynb) Notebook, we can run SQL commands directly against the 3E database. That's great if you know the database structure and are comfortable with it, but what if you are more comfortable running reports and exporting to Excel? We can also run reports directly using 3E's web services.

There are a few libraries in Python that can call web services. I like [Zeep](https://python-zeep.readthedocs.io/en/master/). If you are using virtual environments, activate your virtual environment and install it from the command line with `pip install zeep`. If that previous statement makes no sense you can try installing a package like Zeep directly from within your notebook (I suspect you'll only need to do it once and you can subsequently remove the installation code as you'll get 'already installed' errors). Use [these instructions](https://timonweb.com/tutorials/how-to-install-python-package-from-jupyter-notebook/) to see how. Basically those instructions say to create a Notebook cell like the following cell, however, on my setup running Jupyter Hub as a non-priveledged user with no shell the following fails but it may work for you. I believe `requests` will be installed automatically when `zeep` gets installed, but I'm not 100% certain so we'll also install that package. This can take some time to complete - be patient as long as there is a `*` in the square brackets to the left of the cell.

In [ ]:
import os
import sys
import importlib.util

if not importlib.util.find_spec('zeep'):
    !{sys.executable} -m pip install requests zeep

I imagine that looked a bit nerve racking, but hopefully it installed Zeep and all of it's dependancies. Let's do some more. These next two packages allow us to use the current Windows user identity instead of having to enter in passwords. Don't worry if you got a warning about Pip being out of date.

In [ ]:
if not importlib.util.find_spec('requests_ntlm'):
    !{sys.executable} -m pip install requests_ntlm requests_negotiate_sspi

If you have installed Jupyter Notebook in Linux or if you don't want to run as you, we'll need to specify the credentials we want to use.

This is different (or can be) from the user we used to connect to the SQL server. You can put your own credentials in, or have a system account created with security access to run the processes you wish to execute. It depends on your own and your firm's level of comfort. I generally have an Administrative account with nearly unlimited access I use for this purpose, but if you are beginning, it may be better to use your own credentials against a Dev, QA or UAT instance.

I hide all of these in my environment variables, but feel free to type out your credentials below. Remember, you should be running this on your own machine now. Again, if you are running Notebooks locally on Windows and want to use your own identity, you can set each of these to `None` if you like.

In [ ]:
import os
elite_wapi = os.environ['ELITE_PROD_WAPI']     # the server url like eliteUAT.local or whatever you see in the url when in 3E
elite_instance = os.environ['ELITE_PROD_DB']   # te_3e_uat or whatever your instance is
elite_protocol = 'http'                        # change to https if you have ssl enabled
elite_port = '80'                              # change if you are running on a different port

# if you want to use your own identity:
elite_domain = None
elite_srv_user = None
elite_srv_pass = None

# but if you want to use something else, uncomment these:
# elite_domain = os.environ['ELITE_PROD_DOMAIN'] # the Windows domain of your user
# elite_srv_user = os.environ['ELITE_PROD_SRV_USER'] # the username you'd like to connect with (no domain)
# elite_srv_pass = os.environ['ELITE_PROD_SRV_PASS'] # the password for your user

For subsequent Notebooks we'll have this already installed but as this is the first time we're doing it we need to take care of it now. In future Notebooks, we'll assume we have these packages installed.

I also normally hide all of my 'connect to elite' type functions in a separate file  which gets reused as a class by other Notebooks but for the sake of learning we'll put all of it into this Notebook.

Similar to creating a connection to the SQL database, we need to create a connection to the web services.

In [ ]:
# Import all the necessary libraries
from requests import Session
from requests_ntlm import HttpNtlmAuth
if os.name == 'nt':
    from requests_negotiate_sspi import HttpNegotiateAuth
from zeep import Client
from zeep.transports import Transport

# Establish the connection to Elite using specified or local credentials
wsdl = f"{elite_protocol}://{elite_wapi}:{elite_port}/{elite_instance}/WebUI/Transactionservice.asmx?wsdl"

if elite_domain and elite_srv_user and elite_srv_pass:
    session = Session()
    session.auth = HttpNtlmAuth(f'{elite_domain}\\{elite_srv_user}', elite_srv_pass)
    try:
        elite_client = Client(wsdl, transport=Transport(session=session))
    except:
        raise Exception("Connection to 3E has been severed or could not be established. "
                        "Check the credentials and network connection.")
elif os.name == "nt": # not all credentials exist, try to connect as local user identity
    session = Session()
    session.auth = HttpNegotiateAuth()
    elite_client = Client(wsdl, transport=Transport(session=session))
else:
    raise Exception("You must be on Windows in a Windows network, or provide username, password and domain")

Ok, if you've made it this far we're doing good. Let's run a report. One of the web services offered by 3E is `getdata` which takes in a report query string and returns the results. The trick is to know where to go get the report query string. I'm using 3E 2.8 which has a debug option (I'm not sure if the new HTML5 UI has somethign similar) where I can obtain the string we need.

Let's pick a stock report to use. As an example. Under the Billing Dashboard let's go with **Invoice Master Summary**. Once there, enable the debug pane by right clicking on the 3E logo in the top left and select "Enable Debugging". Pick the **Design** tab. Enter a timekeeper number in the **Billing Timekeeper** field. Add a new field by clicking the **Field+** button and select **Invoice Date**. Select **between** for the comparator and enter a date range like **2019-01-01,2019-08-31**. Run the report.

In the Debug Design tab, press **Refresh** and look for a node called **Presentation**. Expand it by clicking on the little triangle to the left, and click on **Report Query**. In the panel below the Design tree, the report query should be displayed. Click anywhere in there, use `Ctrl`+`A` to select all and `Ctrl`+`C` to copy. That gives you a report query similar to what is shown below on your clipboard.

If were were to use the report query as shown in the design debug window we'll get some errors on xml namespacing. We need to  place the default namespace for the QUERY node to `xmlns="http://elite.com/schemas/query"` in order for the get_data web service call to work correctly (as it is below). Originally it will look something like `<QUERY ID="InvMaster_Q">` but we need it to look like `<QUERY ID="InvMaster_Q" xmlns="http://elite.com/schemas/query">`.

The other thing we do is replace the timekeeper parameter with `{}` so we can pass that in separately if we like. We'll look at that shortly.

In [ ]:
report_query = """
<REPORTQUERY BoundID="InvMaster">
  <REPORTNODE ID="InvMaster" ReportObjectID="InvMaster" ROC="Master Invoice">
    <QUERY ID="InvMaster_Q" xmlns="http://elite.com/schemas/query">
      <SELECT ID="InvMaster_S">
        <OQL_CONTEXT Class="NextGen.Framework.XOQL.OQLContextProvider">
          <NODEMAP Assembly="NextGen.Archetype.InvMaster" Class="NextGen.Application.Query.InvMaster" ID="InvMaster" QueryID="InvMaster" />
        </OQL_CONTEXT>
        <SELECT_LIST>
          <SINGLE_SELECT>
            <NODE NodeID="InvMaster" />
            <WHERE>
              <X_AND_Y>
                <X>
                  <X_IS_EQUAL_TO_Y>
                    <X>
                      <LEAF QueryID="BillTimekeeper1.Number">
                        <NODE NodeID="InvMaster" />
                      </LEAF>
                    </X>
                    <Y>
                      <UNICODE_STRING Value="{}" />
                    </Y>
                  </X_IS_EQUAL_TO_Y>
                </X>
                <Y>
                  <X_IS_BETWEEN_Y_AND_Z>
                    <X>
                      <LEAF QueryID="InvDate">
                        <NODE NodeID="InvMaster" />
                      </LEAF>
                    </X>
                    <Y>
                      <TIME_VALUE Value="2019-01-01T00:00:00.000" />
                    </Y>
                    <Z>
                      <TIME_VALUE Value="2019-08-31T00:00:00.000" />
                    </Z>
                  </X_IS_BETWEEN_Y_AND_Z>
                </Y>
              </X_AND_Y>
            </WHERE>
          </SINGLE_SELECT>
        </SELECT_LIST>
        <ORDER_BY>
          <ORDER Descending="1">
            <LEAF QueryID="InvDate">
              <NODE NodeID="InvMaster" />
            </LEAF>
          </ORDER>
          <ORDER Descending="1">
            <LEAF QueryID="InvNumber">
              <NODE NodeID="InvMaster" />
            </LEAF>
          </ORDER>
        </ORDER_BY>
      </SELECT>
    </QUERY>
  </REPORTNODE>
</REPORTQUERY>
"""

I may have mentioned f-strings and that I really like them for variable substitution in strings. In this case though we'll use the older `format` method of string substitution.

In [ ]:
use_timekeeper_number = '1234'   # pick a timekeeper number from your system
tk_reportquery = report_query.format(use_timekeeper_number)

Does it seem like we took out a timekeeper number, only to put it back in? We did. But we did it in such a way to make re-using the report query over and over a little bit easier. We'd see that if we were to loop through a number of timekeepers to run the same report and handle the results for each. There is an argument to run a report for **all** timekeepers, but my experience with 3E and performance is that you may be better off getting smaller chunks of data back at a time if it can be done that way. It would all depend on your use case.

Which ever the case, lets get the data from that report. We do that with the `GetData` web service method.

In [ ]:
results = elite_client.service.GetData(tk_reportquery)

Alright. Hopefully no errors. The results are in! Let's loop through them and print out a little table. But first, let's take a look at them in their raw form:

In [ ]:
results

You should see a big dump of xml starting with a `<Data xmlns....`. This is the xml response back from 3E. Python has a number of libraries to handle reading xml. We'll use those libraries rather than parsing all that ourselves. Let's write it in a function `parse_3e_xml` which we can improve on later:

In [ ]:
import xml.etree.ElementTree as ET

def parse_3e_xml(xml_data):
    root = ET.fromstring(xml_data)
    data = []
    for timecard in root:
        row_data = {}
        for attributes in timecard:
            for attribute in attributes:
                row_data[attribute.tag.split('}')[1]] = attribute.text
        data.append(row_data)
    return data

The function is defined, let's call it with our results.

In [ ]:
results_parsed = parse_3e_xml(results)

`results_parsed` is now a list of dictionary items. Each item in the list is a row from the report, and the dictionary key value pairs are the columns and row's value. We can take a look at in raw form for reference:

In [ ]:
results_parsed

Now we can loop through and do what ever we'd like to do with the data. We'll demonstrate with a little table.

In [ ]:
# Show a header row
print(f'{"Invoice Number":<20} '
      f'{"Invoice Date":<15} '
      f'{"Fees":>20}')
# and some lines      
print(f'{"-"*20:<20} '
      f'{"-"*15:<15} '
      f'{"-"*20:>20}')
      
running_total = 0.0
for invoice in results_parsed:
    print(f'{invoice["InvNumber"]:<20} '
          f'{invoice["InvDate"]:<15} '
          f'{float(invoice["OrgFee"]):>20,.2f}') # 20 wide, comma separator, 2 decimal
    running_total += float(invoice["OrgFee"])
print(f'{"-"*57:<57}')  # row of lines
print(f'{"Total":<20}  '
      f'{running_total:>35,.2f}')
print(f'{"="*57:<57}')  # row of doublelines

## Conclusion
In this Notebook, we've set up a connection to our Elite WAPI server, called the 3E GetData webservice method and iterated through the results.

-30-